# 👋🌍 Hello, world: Submit a Qiskit job to Quantinuum

In this notebook, we'll review the basics of Azure Quantum by submitting a simple *job*, or quantum program, to [Quantinuum](https://aka.ms/AQ/Quantinuum/Documentation). We will use [Qiskit](https://qiskit.org/) to express the quantum job.

## Submit a simple job to Quantinuum using Azure Quantum
Azure Quantum provides several ways to express quantum programs. In this example we are using Qiskit, but note that Q# and Cirq are also supported. All code in this example will be written in Python.

Let's begin. When you see a code block, hover over it and click the triangle play-button to execute it. To avoid any compilation issues, this should be done in order from top to bottom.

### 1. Connect to the Azure Quantum workspace

To connect to the Azure Quantum service, construct an instance of the `AzureQuantumProvider`. Note that it's imported from `azure.quantum.qiskit`.

In [ ]:
from azure.quantum.qiskit import AzureQuantumProvider

provider = AzureQuantumProvider(
    resource_id = "",
    location = ""
)

Let's see what providers and targets are enabled in this workspace with the following command:

In [ ]:
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor

print("This workspace's targets:")
for backend in provider.backends():
    print("- " + backend.name())

### ❕ Do you see `quantinuum.hqs-lt-s1-apival` in your list of targets? If so, you're ready to keep going.

Don't see it? You may need to add Quantinuum to your workspace to run this sample. Navigate to the **Providers** page in the portal and click **+Add** to add the Quantinuum provider. Don't worry, there's a free credits plan available.
## Quantinuum: The quantum provider
Azure Quantum partners with third-party companies to deliver solutions to quantum jobs. These company offerings are called *providers*. Each provider can offer multiple *targets* with different capabilities. See the table below for Quantinuum's targets.

| Target name | Target ID | Number of qubits | Description |
| --- | --- | --- | --- |
| H1-1 API Validator | `quantinuum.hqs-lt-s1-apival` | 12 | Quantinuum's "API validator." Use this to validate quantum programs before submitting to hardware or emulation on Quantinuum's platform. Free of cost. |
| H1 Emulator | `quantinuum.hqs-lt-s1-sim` | 12 | Quantinuum's quantum simulator. Uses a noise model that is consistent with their H1 hardware offering. |
| Quantinuum System Model: H1 Family | `quantinuum.hqs-lt` | 12 | Quantinuum's H1 family of trapped-ion quantum computers. |

For this example, we will use `quantinuum.hqs-lt-s1-apival` to avoid any costs or credit usage. If you wish to emulate or run the actual circuit, you may replace all instances of `quantinuum.hqs-lt-s1-apival` in subsequent code cells with one of the other values in the table above, but please note any costs incurred. To learn more about Quantinuum's targets, check out our [documentation](https://aka.ms/AQ/Quantinuum/Documentation).

### 2. Build the quantum program

Let's create a simple Qiskit circuit to run.

In [ ]:
# Create a quantum circuit acting on a single qubit
circuit = QuantumCircuit(1,1)
circuit.name = "Single qubit random"
circuit.h(0)
circuit.measure(0, 0)

# Print out the circuit
circuit.draw()

The circuit you built is a simple quantum random bit generator. With Quantinuum's API validator, we will be able to confirm that the circuit is able to be run on their H1 emulator and hardware.

### 3. Submit the quantum program to Quantinuum

In [ ]:
# Create an object that represents Quantinuum's API validator target, "quantinuum.hqs-lt-s1-apival".
#   Note that any target you have enabled in this workspace can
#   be used here. Azure Quantum makes it extremely easy to submit
#   the same quantum program to different providers. 
quantinuum_api_val_backend = provider.get_backend("quantinuum.hqs-lt-s1-apival")

# Using the Quantinuum target target, call "run" to submit the job. We'll
# use a count of 100 (simulated runs).
job = quantinuum_api_val_backend.run(circuit, count=100)
print("Job id:", job.id())

The job ID can be used to retrieve the results later using the [get_details function](https://docs.microsoft.com/en-us/azure/quantum/optimization-job-reference#jobdetails) or by viewing it under the **Job management** section of the portal.

### 4. Obtain the job results
You can monitor the job status with Qiskit's `job_monitor` function.

This may take a minute or so ⏳. Your job will be packaged and sent to Quantinuum, where it will wait its turn to be run. You'll see your job go through these stages: "job is queued" -> "job is actively running" -> "job has successfully run."

In [ ]:
job_monitor(job)

result = job.result()

# The result object is native to the Qiskit package, so we can use Qiskit's tools to print the result as a histogram.
# For API validation, we expect to see all zeroes.
plot_histogram(result.get_counts(circuit), title="Result", number_to_keep=2)

**See the histogram above? Congratulations, you've submitted a job with Azure Quantum! 👏**


### 5. Estimate costs

To estimate the costs of running this program on a simulator or hardware, you can use the `backend.estimate_cost` method.

In [ ]:
backend = provider.get_backend("quantinuum.hqs-lt-s1")
cost = backend.estimate_cost(circuit, shots=100)
print(f"Estimated cost: {cost.estimated_total} {cost.currency_code}")

### 6. Next steps
Next, you can try running a program on Quantinuum's emulation and hardware targets. Just replace `quantinuum.hqs-lt-s1-apival` with `quantinuum.hqs-lt-s1-sim` or `quantinuum.hqs-lt`. Or try another sample by navigating back to the sample gallery. The same "hello world" sample can be run with different quantum providers by choosing another option in the gallery card drop-down menu. Don't worry - your work here is automatically saved.

To learn more about submitting Qiskit circuits to Azure Quantum, review [this documentation](https://docs.microsoft.com/azure/quantum/quickstart-microsoft-qiskit?pivots=platform-quantinuum).

To learn more about job pricing, review [this documentation](https://docs.microsoft.com/azure/quantum/azure-quantum-job-costs).